# Análise de Confiabilidade
### Trabalho Probabilidade Estatística - UniCEUB
### Professor W. Alexandre
### Lucas Gonçalves Balduino | 22409139

In [ ]:
def avaliaExpressao(expressao, prob):
    expressao = expressao.replace(" ", "")
    if len(expressao) == 1:
        return float(prob[expressao])
    tokens = []
    i = 0
    while i < len(expressao):
        ch = expressao[i]
        if ch in "&|":
            tokens.append(ch)
            i += 1
        else:
            tokens.append(ch)
            i += 1
    val = float(prob[tokens[0]])
    idx = 1
    while idx < len(tokens):
        op = tokens[idx]; prox = tokens[idx+1]
        probProx = float(prob[prox])
        if op == "&":
            val = val * probProx
        else:
            val = 1.0 - (1.0 - val) * (1.0 - probProx)
        idx += 2
    return val


def resolve_sistema(sistema, prob):
    
    sistema = sistema.replace(" ", "")
    prob = dict(prob)
    proxLetra = ord("M")
    def criaNovaLetra():
        nonlocal proxLetra
        letra = chr(proxLetra)
        proxLetra += 1
        return letra

    while any(caracter.isdigit() for caracter in sistema):
        stack = []
        reduzido = False
        for i, caracter in enumerate(sistema):
            if caracter.isdigit():
                if stack and stack[-1][0] == caracter:
                    _, start = stack.pop()
                    interno = sistema[start+1:i]
                    if any(c.isdigit() for c in interno):
                        probInterno = resolve_sistema(interno, prob)
                    else:
                        probInterno = avaliaExpressao(interno, prob)
                    letra = criaNovaLetra()
                    prob[letra] = float(probInterno)
                    sistema = sistema[:start] + letra + sistema[i+1:]
                    reduzido = True
                    break
                else:
                    stack.append((caracter, i))
        if not reduzido:
            raise ValueError("Formato inválido")
    if len(sistema) == 1:
        return float(prob[sistema])
    return float(avaliaExpressao(sistema, prob))


## Testes

In [ ]:
testes = [
    ("A", {"A":0.9}),
    ("1A&B1", {"A":0.9, "B":0.8}),
    ("1A|B1", {"A":0.9, "B":0.8}),
    ("1 2 A & B 2 | 3 C 3 1", {"A":0.9, "B":0.8, "C":0.95}),
    ("1 2 A & B 2 & 3 4 C 4 | 5 D 5 3 1", {"A":0.9, "B":0.8, "C":0.95, "D":0.97}),
]

for s, p in testes:
    try:
        r = resolve_sistema(s, p)
        print(s, "->", "{:.6f}".format(r))
    except Exception as e:
        print("Erro em", s, ":", e)


A -> 0.900000
1A&B1 -> 0.720000
1A|B1 -> 0.980000
1 2 A & B 2 | 3 C 3 1 -> 0.986000
1 2 A & B 2 & 3 4 C 4 | 5 D 5 3 1 -> 0.718920
